# 1. Data Open

In [174]:
import re
import numpy as np
import tensorflow as tf
import glob
import os
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion", "I can't get no relief Business men, they drink my wine", 'Plowman dig my earth', 'None were level on the mind', 'Nobody up at his word', 'Hey, hey No reason to get excited', 'The thief he kindly spoke', 'There are many here among us', 'Who feel that life is but a joke', "But, uh, but you and I, we've been through that", 'And this is not our fate', "So let us stop talkin' falsely now", "The hour's getting late, hey All along the watchtower", 'Princes kept the view', 'While all the women came and went', 'Barefoot servants, too', 'Outside in the cold distance', 'A wildcat did growl']


# 2. Data purification.

In [175]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    if len(sentence.split()) > 15: 
        return 0
    return sentence

corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence == 0:
        continue
    corpus.append(preprocessed_sentence)

corpus[:20]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>',
 '<start> who feel that life is but a joke <end>',
 '<start> and this is not our fate <end>',
 '<start> so let us stop talkin falsely now <end>',
 '<start> the hour s getting late , hey all along the watchtower <end>',
 '<start> princes kept the view <end>',
 '<start> while all the women came and went <end>',
 '<start> barefoot servants , too <end>',
 '<start> outside in the cold distance <end>',
 '<start> a wildcat did growl <end>',
 '<start> two riders were approaching <end>']

## 2) Tokenizer 패키지로 텐서로 변환

In [176]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,   # 전체 단어의 개수 
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # padding으로 입력 데이터의 시퀀스 길이를 일정하게 맞추기 (maxlen 15로 설정) 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 271 ...   0   0   0]
 [  2 118   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  45 ...   3   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 633 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f12ff020ed0>


In [177]:
# 변환된 텐서 확인
print(tensor[:3])
tensor.shape

[[   2   62  271   27   94  546   20   86  742   90    3    0    0    0
     0]
 [   2  118    6 6263   10    6 2307    3    0    0    0    0    0    0
     0]
 [   2   62   17  102  184 2711    3    0    0    0    0    0    0    0
     0]]


(156013, 15)

In [178]:
# 단어사전 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [179]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  62 271  27  94 546  20  86 742  90   3   0   0   0]
[ 62 271  27  94 546  20  86 742  90   3   0   0   0   0]


# 3. Separate the dataset

In [180]:
from sklearn.model_selection import train_test_split

# tensor를 train, test 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [181]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [182]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 4. Model design and training.

In [184]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.77245247e-04, -8.58035264e-07, -1.54673573e-04, ...,
         -2.36285967e-04,  3.21980442e-05, -1.63615856e-04],
        [-3.85353400e-04,  1.29268476e-04, -3.38507321e-04, ...,
         -1.65302074e-04,  9.29189264e-05, -2.59378314e-04],
        [-4.14758892e-04,  4.37662937e-04, -3.36711964e-04, ...,
         -7.84124204e-05,  1.26686007e-06, -4.48922598e-04],
        ...,
        [-1.17040111e-03,  1.82645128e-03, -1.70480693e-03, ...,
          6.64320833e-04, -4.49582119e-04, -2.67643132e-03],
        [-1.38644828e-03,  1.92695251e-03, -1.95899373e-03, ...,
          7.63081887e-04, -7.67385063e-04, -2.89417873e-03],
        [-1.59461761e-03,  2.01172032e-03, -2.19243672e-03, ...,
          8.63062218e-04, -1.07874640e-03, -3.08343326e-03]],

       [[-3.77245247e-04, -8.58035264e-07, -1.54673573e-04, ...,
         -2.36285967e-04,  3.21980442e-05, -1.63615856e-04],
        [-7.62600161e-04, -2.05553624e-05, -8

In [185]:
model.summary()

Model: "text_generator_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     multiple                  3072256   
_________________________________________________________________
lstm_20 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_21 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_10 (Dense)             multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


# 4. Model evaluation

In [187]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
609/609 [==============================] - 199s 323ms/step - loss: 3.9136
Epoch 2/10
609/609 [==============================] - 198s 325ms/step - loss: 3.0097
Epoch 3/10
609/609 [==============================] - 198s 325ms/step - loss: 2.8180
Epoch 4/10
609/609 [==============================] - 198s 324ms/step - loss: 2.6731
Epoch 5/10
609/609 [==============================] - 198s 324ms/step - loss: 2.5588
Epoch 6/10
609/609 [==============================] - 198s 324ms/step - loss: 2.4526
Epoch 7/10
609/609 [==============================] - 197s 323ms/step - loss: 2.3601
Epoch 8/10
609/609 [==============================] - 197s 324ms/step - loss: 2.2653
Epoch 9/10
609/609 [==============================] - 197s 323ms/step - loss: 2.1850
Epoch 10/10
609/609 [==============================] - 197s 323ms/step - loss: 2.1126


In [188]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [77]:
# 문장 생성 함수 실행하여 모델에게 작문 시켜보기

In [189]:
generate_text(model, tokenizer, init_sentence="<start> aiffel")

'<start> <unk> <unk> <unk> <unk> <end> '

In [197]:
generate_text(model, tokenizer, init_sentence="<start> happy")

'<start> happy birthday , happy birthday , happy birthday woo , shake ! <end> '

In [191]:
generate_text(model, tokenizer, init_sentence="<start> love")

'<start> love is a beautiful thing <end> '

In [192]:
generate_text(model, tokenizer, init_sentence="<start> computer")

'<start> computer blue <end> '

In [193]:
generate_text(model, tokenizer, init_sentence="<start> sky")

'<start> sky is the limit and you know that you can have <end> '

---
# 회고

+ 1. AIFFEL같은 대중적이지 않은 단어로는 그럴듯한 문장이 생성되지 않았지만
happy, love, sky 등 대중적인 단어로는 학습된 효과로 그럴듯한 문장이 생성되는것을 확이할 수 가 있었다.
+ 2. 데이터 정제부분에서 정규표현식을 이용해서 필요없는 부분을 제거하였고 
15 개 이상이 안되어야하는 문장제거는 start,end 부분을 포함해서 15개이상을 확인하여야 하기때문에 필요없는 부분을 제거하고 마지막 부분에 len(sentence.split()) > 15:  조건문을 추가하여서  진행하였다.   
패딩 같은 경우는 머신러닝에서는 디폴트값도 pre로 되있는데 이번 ex에는 뒤에 0을 추가하는 post 로 되었었다. pre가 더 높은 결과를 보여준다고 되있었는데 이번 모델에서는 별반 차이가 없어서 post로 계속해서 진행을하였다.
+ 3. loss값이 점차 잘 감소하는것을 확인하였고, 2.1로 나쁘지 않은 loss 값이 나왔다.

+ 4. 학습을 마치고나서 다양한 입력단어를 작문을 시켜보았는데 나쁘지 않았던 것 같았다 더 낮은 loss값을 추출하고 더 생소한 단어들을 적더라도 좋은 작문이 될 수 있게 만들어가는것이 좋을것같다